In [12]:
import numpy as np
import xarray as xr
import rioxarray as rxr
from typing import Union
from pathlib import Path

In [2]:
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import dask.array as da

In [17]:
crs = "EPSG:3413"

## Preprocess

Add time bounds with something like
```
for year in {1985..2019}; do
    cdo -O -f nc4 -z zip_2 -settbounds,year -setdate,${year}-1-1 -setreftime,${year}-1-1 GRE_G0240_${year}.nc GRE_G0240_T_${year}.nc; 
done
```

In [3]:
ds = xr.open_mfdataset("../data/itslive/GRE_G0240_T_*.nc", chunks={"x": -1, "y": -1, "time": -1})
vars_to_process = ["v", "vx", "vy", "v_err", "vx_err", "vy_err", "ice"]
ds = ds[vars_to_process]
ice_ds = ds.where(ds["ice"])

In [4]:
jak_ds = ice_ds.sel({"x": slice(-180_000, -80_000), "y": slice(-2_200_000, -2_300_000)})
jak_speed = jak_ds["v"]

In [5]:
nt = ds.time.size
dt = xr.DataArray(
    da.arange(nt, chunks=-1),
    dims=("time"),
)

In [6]:
def idw_weights(distance: xr.DataArray, power: float = 1.0):
    return 1.0 / (distance + 1e-12) ** power    

In [7]:
def save(ds: xr.Dataset, output_filename: Union[str, Path] = "GRE_G0240_1985_2018_IDW_EXP_1.nc" ,comp = dict(zlib=True, complevel=2)
):
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(output_filename, encoding=encoding)


In [8]:
nt = ds.time.size
power = 1

distance = np.isfinite(jak_speed) * dt.broadcast_like(jak_speed)
weights = idw_weights(distance, power=power)

In [9]:
with ProgressBar():
    jak_wm = jak_speed_weighted_mean = jak_ds.weighted(weights).mean(dim="time")
    save(jak_wm, "jak_idw.nc")

[########################################] | 100% Completed | 49.79 s


In [ ]:
with ProgressBar():
    jak_speed_mean = jak_speed.mean(dim="time").compute()

In [34]:
distance = np.isfinite(speed) * dt.broadcast_like(speed)
weights = idw_weights(distance, power=power)
with ProgressBar():
    speed_weighted_mean = ice_ds.weighted(weights).mean(dim="time")
    save(speed_weighted_mean)

[########################################] | 100% Completed | 452.26 s


In [ ]:
speed_weighted_mean.plot(vmax=1500)

In [44]:
ice_ds = ds.where(ds["ice"])
speed = ice_ds["v"]
distance = np.isfinite(speed) * dt.broadcast_like(ice_ds)
weights = idw_weights(distance, power=power)
with ProgressBar():
    weighted_mean = ice_ds.weighted(weights).mean(dim="time").compute()
    weighted_mean.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
    weighted_mean.rio.write_crs(crs, inplace=True)
    weighted_mean = weighted_mean.fillna(0)
#    save(weighted_mean, f"GRE_G0240_1985_2018_IDW_EXP_{power}.nc")

[########################################] | 100% Completed | 400.80 s


In [60]:
g_ds = xr.open_dataset("~/Google Drive/My Drive/Projects/gris-calib/data/2024_05_init_bedmachine/state/gris_g900m_v5_GrIMP_id_CTRL_0_20.nc")

In [61]:
g1800_ds = weighted_mean.interp_like(g_ds).fillna(0)

In [62]:
g1800_ds.rename({"vx": "u_surface_observed", "vy": "v_surface_observed"}).to_netcdf(f"GRE_G900_1985_2018_IDW_EXP_{power}.nc")

In [59]:
(4048.70 * 1) * 1.025

4149.9175

In [63]:
    for v in ["v", "vx", "vy", "v_err", "vx_err", "vy_err"]:
        g1800_ds[v].attrs.update(units="m/yr")


In [30]:
cd notebooks/

/Users/andy/base/pism-ragis/notebooks


/Users/andy/miniforge3/envs/pism-ragis/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [52]:
g1800_ds.v

<xarray.DataArray 'v' (y: 1520, x: 880)> Size: 11MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
    spatial_ref  int64 8B 0
  * y            (y) float64 12kB -3.371e+06 -3.369e+06 ... -6.365e+05
  * x            (x) float64 7kB -6.778e+05 -6.76e+05 ... 9.026e+05 9.044e+05
Attributes:
    units:    m/yr

In [64]:
ls

GRE_G0240_1985_2018_IDW_EXP_1.nc  foo.nc
GRE_G1800_1985_2018_IDW_EXP_1.nc  idw_merge_its_live.ipynb
GRE_G900_1985_2018_IDW_EXP_1.nc   jak_idw.nc


In [65]:
ls ../

#pyproject.toml#                  hpc-systems/
GRE_G1800_1985_2018_IDW_EXP_1.nc  notebooks/
LICENSE                           pism_ragis/
README.md                         pism_ragis.egg-info/
Untitled.ipynb                    plot_results.py
Untitled1.ipynb                   plot_results.py~
VERSION                           pyproject.toml
__pycache__/                      requirements-testing.txt
analysis/                         requirements.txt
config/                           requirements.txt~
data/                             test_systems.ipynb
environment.yml                   tests/
examples/                         uq/
hindcasts/
